<a href="https://colab.research.google.com/github/julianVelandia/GradeWorksUNALDataset/blob/master/Repositorio_UNAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber
!pip install python-certifi-win32
!pip install certifi
!pip install --upgrade certifi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00


In [2]:
import requests
from io import BytesIO
import pdfplumber
import ssl
from bs4 import BeautifulSoup
import json
import os
import re
from typing import List, Dict

In [3]:
def save_uri_in_json(uri):
  file_name = 'uris.json'
  if os.path.exists(file_name):
      with open(file_name, 'r', encoding='utf-8') as json_file:
          data = json.load(json_file)
  else:
      data = {"uris": {}}


  data["uris"][uri] = "pending"

  with open(file_name, 'w', encoding='utf-8') as json_file:
      json.dump(data, json_file, indent=4, ensure_ascii=False)

In [4]:
def save_metadata_in_json(uri: str, data_dict: Dict[str, dict]):
    file_name = 'dataset.json'

    try:
        try:
            with open(file_name, 'r', encoding='utf-8') as json_file:
                existing_data = json.load(json_file)
        except (FileNotFoundError, json.JSONDecodeError):
            existing_data = {}


        existing_data[uri] = data_dict

        with open(file_name, 'w',  encoding='utf-8') as json_file:
            json.dump(existing_data, json_file, indent=4, ensure_ascii=False)

    except Exception as e:
        print(f"ERROR: {e}")

In [5]:
def save_uris_pdfs_and_metadata_from_repositorio_unal():
  base_url = "https://repositorio.unal.edu.co/handle/unal/5/recent-submissions?offset="
  offset = 0
  url_list = []
  while offset <= 1900:
      url = base_url + str(offset)
      print('OFFSET: '+ str(offset))
      offset += 20
      try:
        response = requests.get(url, verify=False)
        soup = BeautifulSoup(response.content, 'html.parser')

        artifact_items = soup.find_all(class_="ds-artifact-item")

        for item in artifact_items:
            item_wrapper = item.find('div', class_="item-wrapper row")
            if item_wrapper:
                col_div = item_wrapper.find('div', class_="col-sm-3 hidden-xs")

                if col_div:
                    thumbnail_div = col_div.find('div', class_="thumbnail artifact-preview")
                    if thumbnail_div:
                        a_tag = thumbnail_div.find('a', class_="image-link")

                        if a_tag and 'href' in a_tag.attrs:
                            item_page_url = a_tag['href']
                            item_page_url = item_page_url.strip()
                            item_page_url = 'https://repositorio.unal.edu.co' + item_page_url + '?show=full'
                            item_page_response = requests.get(item_page_url, verify=False)
                            item_page_soup = BeautifulSoup(item_page_response.content, 'html.parser')

                            advisor_label = item_page_soup.find('td', class_='label-cell', string='dc.contributor.advisor')
                            if advisor_label:
                                advisor_row = advisor_label.find_parent('tr')
                                advisor = str(advisor_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')


                            author_label = item_page_soup.find('td', class_='label-cell', string='dc.contributor.author')
                            if author_label:
                                author_row = author_label.find_parent('tr')
                                author = str(author_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')


                            date_label = item_page_soup.find('td', class_='label-cell', string='dc.date.issued')
                            if date_label:
                                date_row = date_label.find_parent('tr')
                                date = str(date_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')

                            description_label = item_page_soup.find('td', class_='label-cell', string='dc.description.abstract')
                            if description_label:
                                description_row = description_label.find_parent('tr')
                                description = str(description_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')

                            title_label = item_page_soup.find('td', class_='label-cell', string='dc.title')
                            if title_label:
                                title_row = title_label.find_parent('tr')
                                title = str(title_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')


                            program_label = item_page_soup.find('td', class_='label-cell', string='dc.publisher.program')
                            if program_label:
                                program_row = program_label.find_parent('tr')
                                program = str(program_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')


                            faculty_label = item_page_soup.find('td', class_='label-cell', string='dc.publisher.faculty')
                            if faculty_label:
                                faculty_row = faculty_label.find_parent('tr')
                                faculty = str(faculty_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')


                            data_dict = {
                                "advisor": advisor,
                                "author": author,
                                "date": date,
                                "description": description,
                                "title": title,
                                "program": program,
                                "faculty": faculty
                            }
                            url_element = item_page_soup.select_one('.thumbnail a')
                            url_key =''
                            if url_element:
                                url_key = url_element.get('href')
                            else:

                              continue


                            save_uri_in_json(url_key)
                            save_metadata_in_json(url_key, data_dict)



      except Exception as e:
        print('ERROR: '+url + str(e))




<img src="https://github.com/user-attachments/assets/a81d8a9d-4112-4ddd-bec8-325435c21c2a" alt="image" width="600"/>


In [6]:
  if not os.path.exists('dataset.json') and not os.path.exists('uris.json'):
      save_uris_pdfs_and_metadata_from_repositorio_unal()


Se deberían tener 2 archivos:
- Dataset.json
 - Lista de la información preliminar de la tesis (advisor, author, date, description, rtitle, program, faculty), con la uri como llave
- uris.json
 - Lista de las uris (que son las keys de dataset) y un estado pending


- Dataset.json

```json
{
    "/bitstream/handle/unal/84578/1053812256.2023.pdf?sequence=2&isAllowed=y": {
        "advisor": "Herrera León, Fernando Augusto",
        "author": "Téllez González, Jonathan Salvador",
        "date": "2023-08-11",
        "description": "La finalidad del trabajo presente es mostrar ...",
        "title": "Diseño de iluminación, control y embellecimiento de la cancha de Futbol del Estadio Alfonso López",
        "program": "Bogotá - Ingeniería - Especialización en Iluminación Pública y Privada",
        "faculty": "Facultad de Ingeniería"
}
```

- uris.json

```json
{
    "uris": {
        "/bitstream/handle/unal/84638/46386566.2023.pdf?sequence=2&isAllowed=y": "pending",
        "/bitstream/handle/unal/84578/1053812256.2023.pdf?sequence=2&isAllowed=y": "pending",
}
```

In [7]:
def from_url_to_txt(pdf_url):
  try:
    response = requests.get(pdf_url, verify=False)
    response.raise_for_status()
    pdf_stream = pdfplumber.open(BytesIO(response.content))
    pdf_text = ""
    for page in pdf_stream.pages:
      page_text = page.extract_text()
      pdf_text += page_text
    pdf_stream.close()
  except:
    print('ERROR: '+pdf_url)
  return pdf_text

In [8]:
def save_content_in_json(uri, raw_content):
  file_name = 'raw_dataset.json'
  if os.path.exists(file_name):
      with open(file_name, 'r', encoding='utf-8') as json_file:
          data = json.load(json_file)
  else:
      data = {"raw": []}


  data["raw"].append({"uri": uri, "raw_content": raw_content})

  with open(file_name, 'w', encoding='utf-8') as json_file:
      json.dump(data, json_file, indent=4, ensure_ascii=False)

In [9]:
def update_uri_state(uri, new_state):
    file_name = 'uris.json'
    if os.path.exists(file_name):
        with open(file_name, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
        if uri in data["uris"]:
            data["uris"][uri] = new_state
            with open(file_name, 'w', encoding='utf-8') as json_file:
                json.dump(data, json_file, indent=4, ensure_ascii=False)

In [10]:
def get_pending_uris():
    file_name = 'uris.json'
    pending_uris = []

    if not os.path.exists(file_name):
      uri_list = save_uris_pdfs_and_metadata_from_repositorio_unal()
      return uri_list

    with open(file_name, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
        uris_dict = data.get("uris", {})

    for uri, state in uris_dict.items():
        if state == "pending":
            pending_uris.append(uri)

    return pending_uris

def get_error_uris():
    file_name = 'uris.json'
    error_uris = []

    with open(file_name, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
        uris_dict = data.get("uris", {})

    for uri, state in uris_dict.items():
        if state == "error":
            error_uris.append(uri)

    return error_uris

In [18]:
uri_list = get_pending_uris()

In [ ]:
len(get_error_uris())

In [ ]:
len(uri_list)

In [21]:
def read_pdfs_by_uri_and_update_state():
  for uri in uri_list:
    print('INICIA uri: ' + uri)
    url = 'https://repositorio.unal.edu.co'+uri
    try:
      raw_content = from_url_to_txt(url)
    except:
      update_uri_state(uri, 'error')

    if raw_content != '':
      print('FINALIZA uri: ' + uri+ ' len: '+ str(len(raw_content)))
      save_content_in_json(uri, raw_content)
      update_uri_state(uri, 'complete')
    else:
      print('ERROR URI: '+ uri)
      update_uri_state(uri, 'error')


In [22]:
read_pdfs_by_uri_and_update_state()

In [23]:
def get_list_raw_dataset():
  file_name = 'raw_dataset.json'
  raw_dataset = []
  with open(file_name, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

  return data

In [24]:
def join_raw_data_to_dataset():
  with open('dataset.json', 'r', encoding='utf-8') as dataset_file:
    dataset = json.load(dataset_file)

  with open('raw_dataset.json', 'r', encoding='utf-8') as raw_dataset_file:
      raw_dataset = json.load(raw_dataset_file)

  raw_content_mapping = {entry['uri']: entry['raw_content'] for entry in raw_dataset['raw']}

  for uri, entry in dataset.items():
      if uri in raw_content_mapping:
          entry['raw_content'] = raw_content_mapping[uri]

  with open('dataset.json', 'w', encoding='utf-8') as updated_dataset_file:
      json.dump(dataset, updated_dataset_file, indent=4, ensure_ascii=False)

In [ ]:
join_raw_data_to_dataset()

<img src="https://github.com/user-attachments/assets/1850b8fc-eb75-4d4c-9e31-70a21fde1a18" alt="image" width="600"/>



- dataset.json

```json
{
    "/bitstream/handle/unal/84578/1053812256.2023.pdf?sequence=2&isAllowed=y": {
        "advisor": "Herrera León, Fernando Augusto",
        "author": "Téllez González, Jonathan Salvador",
        "date": "2023-08-11",
        "description": "La finalidad del trabajo presente es mostrar ...",
        "title": "Diseño de iluminación, control y embellecimiento de la cancha de Futbol del Estadio Alfonso López",
        "program": "Bogotá - Ingeniería - Especialización en Iluminación Pública y Privada",
        "faculty": "Facultad de Ingeniería",
        "raw_content": "Diseño de iluminación, control y\nembellecimiento de la cancha de\nFutbol..."
    }
}
```

- uris.json

```json
{
    "uris": {
        "/bitstream/handle/unal/84638/46386566.2023.pdf?sequence=2&isAllowed=y": "pending",
        "/bitstream/handle/unal/84578/1053812256.2023.pdf?sequence=2&isAllowed=y": "pending"
    }
}
```

- raw_dataset.json

```json
{
    "raw": [
        {
            "uri": "/bitstream/handle/unal/84578/1053812256.2023.pdf?sequence=2&isAllowed=y",
            "raw_content": "Diseño de iluminación, control y\nembellecimiento de la cancha de\nFutbol..."
        }
    ]
}
```

In [26]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [30]:
from datasets import Dataset
from huggingface_hub import HfApi
import json

def upload_individual_datasets(base_name, files, hf_token):

    api = HfApi()
    user = api.whoami(token=hf_token)

    for file_name in files:
        try:
            with open(file_name, "r", encoding="utf-8") as f:
                data = json.load(f)
        except json.JSONDecodeError as e:
            print(f"ERROR")
            continue

        dataset_name_key = file_name.split('.')[0]
        dataset = Dataset.from_dict({"content": [data]})

        full_dataset_name = f"{base_name}-{dataset_name_key}"

        dataset.push_to_hub(full_dataset_name, token=hf_token)
        print(f"Dataset '{full_dataset_name}' OK")

base_name = "unal-repository-dataset"
files = ["dataset.json", "uris.json", "raw_dataset.json"]
hf_token = ":)"

upload_individual_datasets(base_name, files, hf_token)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/875k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Dataset 'unal-repository-dataset-dataset' OK


...

No files have been modified since last commit. Skipping to prevent empty commit.


Dataset 'unal-repository-dataset-uris' OK


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset 'unal-repository-dataset-raw_dataset' OK


In [32]:
import json
import pandas as pd

with open("uris.json", "r", encoding="utf-8") as f:
    uris_data = json.load(f)

uris_df = pd.DataFrame(list(uris_data["uris"].items()), columns=["URI", "Estado"])
print("Información general de uris.json:")
print(uris_df.info())
print("\nConteo por estado:")
print(uris_df["Estado"].value_counts())
print("\nPrimeras filas:")
print(uris_df.head())


Información general de uris.json:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1908 entries, 0 to 1907
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URI     1908 non-null   object
 1   Estado  1908 non-null   object
dtypes: object(2)
memory usage: 29.9+ KB
None

Conteo por estado:
Estado
complete    1908
Name: count, dtype: int64

Primeras filas:
                                                 URI    Estado
0  /bitstream/handle/unal/84638/46386566.2023.pdf...  complete
1  /bitstream/handle/unal/84578/1053812256.2023.p...  complete
2  /bitstream/handle/unal/84464/1085925944.2023.p...  complete
3  /bitstream/handle/unal/84427/Trabajo%20Final%2...  complete
4  /bitstream/handle/unal/84419/LineamientosGeotu...  complete


In [33]:
with open("raw_dataset.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

raw_df = pd.DataFrame(raw_data["raw"])
print("Información general de raw_dataset.json:")
print(raw_df.info())
print("\nPrimeras filas:")
print(raw_df.head())
print("\nDistribución de longitud del contenido:")
raw_df["raw_content_length"] = raw_df["raw_content"].apply(len)
print(raw_df["raw_content_length"].describe())


Información general de raw_dataset.json:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1884 entries, 0 to 1883
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uri          1884 non-null   object
 1   raw_content  1884 non-null   object
dtypes: object(2)
memory usage: 29.6+ KB
None

Primeras filas:
                                                 uri  \
0  /bitstream/handle/unal/84578/1053812256.2023.p...   
1  /bitstream/handle/unal/84464/1085925944.2023.p...   
2  /bitstream/handle/unal/84427/Trabajo%20Final%2...   
3  /bitstream/handle/unal/84419/LineamientosGeotu...   
4  /bitstream/handle/unal/84411/1.098.633.701.202...   

                                         raw_content  
0  Diseño de iluminación, control y\nembellecimie...  
1  Aplicación de los criterios AMOS\npara la clas...  
2  Diseño de iluminación del Piso 2\nUnidad de Cu...  
3  Lineamientos para el Geoturismo de\nvolcanes e...  
4  Factores as

In [34]:
import json
import pandas as pd

with open("dataset.json", "r", encoding="utf-8") as f:
    dataset_data = json.load(f)

dataset_df = pd.DataFrame.from_dict(dataset_data, orient="index")
print("Información general de dataset.json:")
print(dataset_df.info())
print("\nPrimeras filas:")
print(dataset_df.head())
print("\nEstadísticas descriptivas:")
print(dataset_df.describe(include="all"))


Información general de dataset.json:
<class 'pandas.core.frame.DataFrame'>
Index: 1910 entries, /bitstream/handle/unal/84578/1053812256.2023.pdf?sequence=2&isAllowed=y to /bitstream/handle/unal/2400/0502042.2007.pdf?sequence=1&isAllowed=y
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   advisor      1910 non-null   object
 1   author       1910 non-null   object
 2   date         1910 non-null   object
 3   description  1910 non-null   object
 4   title        1910 non-null   object
 5   program      1910 non-null   object
 6   faculty      1910 non-null   object
 7   raw_content  1859 non-null   object
dtypes: object(8)
memory usage: 134.3+ KB
None

Primeras filas:
                                                                              advisor  \
/bitstream/handle/unal/84578/1053812256.2023.pd...     Herrera León, Fernando Augusto   
/bitstream/handle/unal/84464/1085925944.2023.pd...    Acosta Forero, Bernarda